In [1]:

from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from statsmodels.tsa import stattools
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import *

from sklearn.datasets import load_digits
from scipy.io import arff

from sklearn.feature_selection import SelectKBest, chi2

In [2]:
######################## ne pas run si vous possedez dfFPCA dans le répertoire


dfL = pd.read_csv('WADI_attackdataLABLE.csv',header=1) #contient NON-ATTACK ET ATTACK

dfL = pd.concat([dfL[dfL.label == 1],dfL[dfL.label == -1]],axis=0)

dfL.dropna(subset =['Time'], inplace=True)
dfL = dfL.fillna(0)


df = pd.read_csv("WADI_14days_new.csv") # contient seulement NON-ATTACK
#df = df[df.Row < 500000]
print(len(df))

df = df.sample(n=500000)


df = df.fillna(0) # on pourrait aussi remplacer par une valeur moyenne

## essaie 1

dfL.columns = np.append(df.columns.values,'label')

dtempolabel = pd.DataFrame( data = np.full(len(df),1) , columns = ['label']   )
dtempolabel.reset_index()
df.reset_index()


df = pd.DataFrame( data = df.values , columns = df.columns.values  )

dfF =  pd.concat([ pd.concat([df  , dtempolabel ],axis=1) , dfL ],axis=0)

df=None
dfL=None
##


784571


In [3]:
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer(output_distribution = "normal")

dfF[dfF.columns.values[3:-1]] = scaler.fit_transform(dfF[dfF.columns.values[3:-1]])

list_ndiscrt = list(dfF.columns.values)
list_ndiscrt.remove('Row')
list_ndiscrt.remove('Date')
list_ndiscrt.remove('Time')
list_ndiscrt.remove('label')






In [4]:

from sklearn.decomposition import PCA

pca = PCA(n_components = 10)
Y = dfF.drop(list_ndiscrt,axis=1)
principalComponents = pca.fit_transform(dfF.drop(['Row', 'Date', 'Time','label'],axis=1))

dfF = pd.concat([pd.DataFrame(data = principalComponents),pd.DataFrame(data = Y.values,columns=Y.columns.values)],axis=1)

principalComponents=None
Y=None
pca=None


In [ ]:
dfF.to_csv('dfPCA')

#FIN de ne pas run

In [5]:
dfF = pd.read_csv("dfPCA").drop(['Unnamed: 0'],axis=1)

In [6]:
dfF

,0,1,2,3,4,5,6,7,8,9,Row,Date,Time,label
0,-14.001313,3.066117,1.074653,0.461158,-0.622662,-1.365114,0.339726,1.374506,0.568657,0.330154,55768.0,9/26/2017,29:27.0,1
1,8.142228,-9.441209,-7.187098,0.223193,-3.764465,0.511003,-0.266641,1.085530,-0.932413,-1.018362,615472.0,10/2/17,57:51.0,1
2,-6.866756,-3.359628,-2.044650,-1.100442,-0.120884,6.296805,-3.088411,0.796094,8.116797,2.702922,19684.0,9/25/2017,28:03.0,1
3,-3.361627,2.705087,-14.443270,-1.069469,0.207831,-0.221929,5.878527,-1.698635,1.485816,2.140105,884789.0,10/5/17,46:28.0,1
4,-7.729583,-2.873254,-2.856369,-1.484573,0.433607,7.637214,-4.921058,1.824469,8.960950,0.797873,730367.0,10/4/17,52:46.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672796,0.921716,0.784559,15.340798,-4.829226,13.041172,4.256880,-0.574622,1.406076,-1.806730,-1.782467,164220.0,10/11/17,36:59.0,-1
672797,-2.084234,-0.315809,13.497337,-11.426457,3.244842,4.466527,-0.534882,1.871791,-1.986519,-0.406665,164221.0,10/11/17,37:00.0,-1
672798,-2.084234,-0.315809,13.497337,-11.426457,3.244842,4.466527,-0.534882,1.871791,-1.986519,-0.406665,164222.0,10/11/17,37:01.0,-1
672799,-2.084234,-0.315809,13.497337,-11.426457,3.244842,4.466527,-0.534882,1.871791,-1.986519,-0.406665,164223.0,10/11/17,37:02.0,-1


In [7]:

dfSmaller =  dfF[dfF.label == 1]
data_train = dfSmaller.sample(frac=0.7)
data_test = dfSmaller.drop(data_train.index)
data_testAttack = dfF[dfF.label == -1]
len(dfSmaller)


662824

In [8]:

from sklearn.neighbors import LocalOutlierFactor
print("debut fit")
clf = LocalOutlierFactor(n_neighbors=20, n_jobs=-1 , novelty=True, contamination='auto',metric='braycurtis').fit(data_train.drop(['Row', 'Date', 'Time','label'],axis=1))

res = clf.predict(data_test.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs normales local outlier")
print(pd.Series(res).value_counts())

res = clf.predict(data_testAttack.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs Anormales local outlier")
print(pd.Series(res).value_counts())

debut fit
prédiction valeurs normales local outlier
 1    184919
-1     13928
dtype: int64
prédiction valeurs Anormales local outlier
-1    8183
 1    1794
dtype: int64


In [10]:
 ((100-13928/(184919+13928)*100)+(100-1794/(8183+1794)*100))/2 # score pour lof

87.50713131323357

In [9]:
# chebyshev pas terrible
from sklearn.neighbors import LocalOutlierFactor
print("debut fit")
clf = LocalOutlierFactor(n_neighbors=20, n_jobs=-1 , novelty=True, contamination='auto').fit(data_train.drop(['Row', 'Date', 'Time','label'],axis=1))

res = clf.predict(data_test.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs normales local outlier")
print(pd.Series(res).value_counts())

res = clf.predict(data_testAttack.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs Anormales local outlier")
print(pd.Series(res).value_counts())

debut fit
prédiction valeurs normales local outlier
 1    184759
-1     14088
dtype: int64
prédiction valeurs Anormales local outlier
-1    7694
 1    2283
dtype: int64


In [19]:

dfSmaller =  dfF[dfF.label == 1].sample(n=60000)
data_train = dfSmaller.sample(frac=0.7)
data_test = dfSmaller.drop(data_train.index)
data_testAttack = dfF[dfF.label == -1]
len(dfSmaller)


60000

In [15]:

from sklearn.neighbors import LocalOutlierFactor
print("debut fit")
clf = LocalOutlierFactor(n_neighbors=20, n_jobs=-1 , novelty=True, contamination='auto',metric='hamming').fit(data_train.drop(['Row', 'Date', 'Time','label'],axis=1))

res = clf.predict(data_test.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs normales local outlier")
print(pd.Series(res).value_counts())

res = clf.predict(data_testAttack.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs Anormales local outlier")
print(pd.Series(res).value_counts())

debut fit
prédiction valeurs normales local outlier
1    9000
dtype: int64
prédiction valeurs Anormales local outlier
1    9977
dtype: int64


In [20]:

from sklearn.neighbors import LocalOutlierFactor
print("debut fit")
clf = LocalOutlierFactor(n_neighbors=20, n_jobs=-1 , novelty=True, contamination='auto',metric='sqeuclidean').fit(data_train.drop(['Row', 'Date', 'Time','label'],axis=1))

res = clf.predict(data_test.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs normales local outlier")
print(pd.Series(res).value_counts())

res = clf.predict(data_testAttack.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs Anormales local outlier")
print(pd.Series(res).value_counts())

debut fit
prédiction valeurs normales local outlier
 1    13124
-1     4876
dtype: int64
prédiction valeurs Anormales local outlier
-1    7228
 1    2749
dtype: int64


In [ ]:
#mahalanobis fonctionne pas

In [21]:

from sklearn.neighbors import LocalOutlierFactor
print("debut fit")
clf = LocalOutlierFactor(n_neighbors=20, n_jobs=-1 , novelty=True, contamination='auto',metric='cosine').fit(data_train.drop(['Row', 'Date', 'Time','label'],axis=1))

res = clf.predict(data_test.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs normales local outlier")
print(pd.Series(res).value_counts())

res = clf.predict(data_testAttack.drop(['Row', 'Date', 'Time','label'],axis=1))
print("prédiction valeurs Anormales local outlier")
print(pd.Series(res).value_counts())

debut fit
prédiction valeurs normales local outlier
 1    13109
-1     4891
dtype: int64
prédiction valeurs Anormales local outlier
-1    7078
 1    2899
dtype: int64


In [ ]:

from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn import ensemble

data_train = dfF.sample(frac=0.7)
data_test = dfF.drop(data_train.index)


Y_train = data_train['label'].values
X_train = data_train.drop(['Row', 'Date', 'Time','label'],axis=1).values




Y_test = data_test['label'].values
X_test = data_test.drop(['Row', 'Date', 'Time','label'],axis=1).values


print(print(pd.Series(Y_test).value_counts()))

In [27]:
from sklearn import tree
model = ensemble.RandomForestClassifier()
model.fit(X_train,  Y_train)

y_dct = model.predict(X_test)
print(accuracy_score(Y_test,   y_dct))
from sklearn.metrics import f1_score
print(f1_score(Y_test, y_dct, average=None))

0.999963324959199
[0.99770642 0.99998151]


In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,y_dct)

array([[   870,      1],
       [     3, 108192]], dtype=int64)

In [32]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier()
model.fit(X_train,  Y_train)

y_dct = model.predict(X_test)
print(accuracy_score(Y_test,   y_dct))
from sklearn.metrics import f1_score
print(f1_score(Y_test, y_dct, average=None))

0.9938971997620569
[0.93958561 0.99678628]


In [29]:
from sklearn import tree
model = GradientBoostingClassifier()
model.fit(X_train,  Y_train)

y_dct = model.predict(X_test)
print(accuracy_score(Y_test,   y_dct))
from sklearn.metrics import f1_score
print(f1_score(Y_test, y_dct, average=None))

0.9992573304237801
[0.95152603 0.9996258 ]


In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,y_dct)

array([[   795,     76],
       [     5, 108190]], dtype=int64)

In [29]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train,  Y_train)

y_dct = model.predict(X_test)
print(accuracy_score(Y_test,   y_dct))
from sklearn.metrics import f1_score
print(f1_score(Y_test, y_dct, average=None))

0.9927515477318293
[0.9280245 0.9961836]


In [30]:
from sklearn.ensemble import  ExtraTreesClassifier
modelerf =  ExtraTreesClassifier()
modelerf.fit(X_train,  Y_train)

y_dct = modelerf.predict(X_test)
print(accuracy_score(Y_test,   y_dct))
from sklearn.metrics import f1_score
print(f1_score(Y_test, y_dct, average=None))




0.9925091982638965
[0.92427617 0.99605971]


In [31]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(X_train,  Y_train)

y_dct = model.predict(X_test)
print(accuracy_score(Y_test,   y_dct))
from sklearn.metrics import f1_score
print(f1_score(Y_test, y_dct, average=None))

0.9871554781995637
[0.86551326 0.99325567]


In [ ]:
###################################################################################################################
## cross val

In [35]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier()
ss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
ss.get_n_splits(X_train, Y_train)
for train_index, test_index in ss.split(X_train, Y_train):
    trainx, testx = X_train[train_index], X_train[test_index]
    trainy, testy = Y_train[train_index], Y_train[test_index]
    model.fit(trainx,  trainy)
    y_dct = model.predict(testx)
    print(f1_score(testy, y_dct, average=None))
y_dct = model.predict(X_test)

print('final score : ',accuracy_score(Y_test,   y_dct))
from sklearn.metrics import f1_score
print(f1_score(Y_test, y_dct, average=None))

[0.99634523 0.99986793]
[0.99606631 0.99985776]
[0.99705015 0.99989332]
[0.99522472 0.99982728]
[0.99579596 0.99984759]
final score :  0.999836957629364
[0.99556049 0.99991695]


In [36]:
from sklearn.model_selection import StratifiedShuffleSplit
model =  ExtraTreesClassifier()
ss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
ss.get_n_splits(X_train, Y_train)
for train_index, test_index in ss.split(X_train, Y_train):
    trainx, testx = X_train[train_index], X_train[test_index]
    trainy, testy = Y_train[train_index], Y_train[test_index]
    model.fit(trainx,  trainy)
    y_dct = model.predict(testx)
    print(f1_score(testy, y_dct, average=None))
    
y_dct = model.predict(X_test)

print('final score : ',accuracy_score(Y_test,   y_dct))
from sklearn.metrics import f1_score
print(f1_score(Y_test, y_dct, average=None))

[0.99564546 0.99984252]
[0.99423915 0.99979172]
[0.99592525 0.99985268]
[0.99634523 0.99986793]
[0.9962084  0.99986284]
final score :  0.9998980985183524
[0.99722684 0.9999481 ]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=2)
ss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
ss.get_n_splits(X_train, Y_train)
for train_index, test_index in ss.split(X_train, Y_train):
    trainx, testx = X_train[train_index], X_train[test_index]
    trainy, testy = Y_train[train_index], Y_train[test_index]
    model.fit(trainx,  trainy)
    y_dct = model.predict(testx)
    print(f1_score(testy, y_dct, average=None))
    
y_dct = model.predict(X_test)

print('final score : ',accuracy_score(Y_test,   y_dct))
from sklearn.metrics import f1_score
print(f1_score(Y_test, y_dct, average=None))